In [1]:

import numpy as np
import os
import time
import cv2
import pandas as pd 
import matplotlib.pyplot as plt
from math import *
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

import argparse


In [2]:
mainPath="F:\IBM\TRAIN_SET"
destPath="F:\IBM\DEST"
    

im_size = 384
rangeLower=0
rangeUpper=100
imgsInFile=30
startFileCount=0
augmentCount=5

In [3]:
apples=995
b=1374
o=1019
pa=275
wm=475

classId=["APPLES","BANANA","ORANGE","PINEAPPLE","WATERMELON"]


In [4]:
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder , OneHotEncoder

inputs = layers.Input(shape=(im_size, im_size, 3))
size = (im_size, im_size)

noOfClasses = 5
imgCategory=np.array([i for i in range(noOfClasses)])
inputImg=[]
imgLabels=[]

    
for i in range(len(classId)):
    dirPath=mainPath+"\\"+str(classId[i])
    for j in os.listdir(dirPath): 
        ImgPath=mainPath+"\\"+str(classId[i])+"\\"+j 
        image = cv2.imread(ImgPath)       
        if(image is not None):
            #BGR->RGB
            img2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            img3 = cv2.resize(img2, (im_size, im_size))
            img3 = img3.astype('float32') / 255.0       
            inputImg.append(img3)
            imgLabels=imgLabels+[i]
        
    

In [5]:
data=np.array(inputImg[:60]+inputImg[apples:apples+60]+inputImg[apples+b:apples+b+60]+inputImg[apples+b+o:apples+b+o+60]+inputImg[apples+b+o+pa:apples+b+o+pa+60])
labels=np.array(imgLabels[:60]+imgLabels[apples:apples+60]+imgLabels[apples+b:apples+b+60]+imgLabels[apples+b+o:apples+b+o+60]+imgLabels[apples+b+o+pa:apples+b+o+pa+60])
df = pd.DataFrame(labels,columns=["A"])

print(df)
print(type(df))
one_hot_encoded_data = pd.get_dummies(df,columns=["A"]).to_numpy()
print(one_hot_encoded_data)

     A
0    0
1    0
2    0
3    0
4    0
..  ..
295  4
296  4
297  4
298  4
299  4

[300 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 ...
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]]


In [6]:


print(data.shape)
print(labels.shape)
print(data.size)
print(labels.size)
print(len(inputImg))
print(len(imgLabels))
print(one_hot_encoded_data.shape)

(300, 384, 384, 3)
(300,)
132710400
300
4138
4138
(300, 5)


In [7]:
outputs = EfficientNetV2S(include_top=True, weights=None, classes=noOfClasses)(inputs)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"] )
model.summary()
model.fit(data, one_hot_encoded_data, epochs=1, batch_size=30)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 384, 384, 3)]     0         
                                                                 
 efficientnetv2-s (Functiona  (None, 5)                20337765  
 l)                                                              
                                                                 
Total params: 20,337,765
Trainable params: 20,183,893
Non-trainable params: 153,872
_________________________________________________________________


KeyboardInterrupt: 

In [ ]:
if not(os.path.exists('./WeightsDir'+str(fcnt))):
    os.makedirs('./WeightsDir'+str(fcnt))
model.save_weights('./WeightsDir'+str(fcnt)+'/weights')
    